In [65]:
# 평가 지표 평균 90 이상
import pandas as pd
from numpy import mean, std
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from imblearn.under_sampling import TomekLinks
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from imblearn.under_sampling import *

df = pd.read_csv('./TravelInsurancePrediction.csv')
df['FrequentFlyer'] = df['FrequentFlyer'].map({'Yes': 1, 'No': 0})
df['EverTravelledAbroad'] = df['EverTravelledAbroad'].map({'Yes': 1, 'No': 0})
df["Employment Type"] = df["Employment Type"].map({"Government Sector" : 1, "Private Sector/Self Employed" : 0})
df['GraduateOrNot'] = df['GraduateOrNot'].map({'Yes': 1, 'No': 0})

# df = df.drop(["GraduateOrNot"] , axis=1)
df_index = ['Age', 'AnnualIncome', 'ChronicDiseases',"GraduateOrNot", 
          'FrequentFlyer', 'FamilyMembers', 'Employment Type',
          'EverTravelledAbroad']
ndf = df[['Age', 'AnnualIncome', 'ChronicDiseases', 'GraduateOrNot',
          'FrequentFlyer', 'FamilyMembers', 'Employment Type',
          'EverTravelledAbroad', 'TravelInsurance']]
X = ndf[list(ndf.columns)[:-1]]
y = ndf[list(ndf.columns)[-1]]
X = preprocessing.StandardScaler().fit(X).transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=10)
print(X_train.shape)

(1589, 8)


#### 언더샘플링

In [66]:
df_Xnew = pd.DataFrame(X_train, columns = df_index)
df_ynew = pd.DataFrame(y_train, columns = ['TravelInsurance'])

df_ynew.reset_index(drop=True,inplace = True)
df_new = pd.concat([df_Xnew, df_ynew], axis = 1)
df_new.info()
ndf1 = df_new[['Age', 'AnnualIncome', 'ChronicDiseases', 'GraduateOrNot',
          'FrequentFlyer', 'FamilyMembers', 'Employment Type',
          'EverTravelledAbroad', 'TravelInsurance']]

X  = ndf1[list(ndf1.columns)[:-1]]
y  = ndf1[list(ndf1.columns)[-1]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1589 entries, 0 to 1588
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  1589 non-null   float64
 1   AnnualIncome         1589 non-null   float64
 2   ChronicDiseases      1589 non-null   float64
 3   GraduateOrNot        1589 non-null   float64
 4   FrequentFlyer        1589 non-null   float64
 5   FamilyMembers        1589 non-null   float64
 6   Employment Type      1589 non-null   float64
 7   EverTravelledAbroad  1589 non-null   float64
 8   TravelInsurance      1589 non-null   int64  
dtypes: float64(8), int64(1)
memory usage: 111.9 KB


In [67]:
# 평균 73, 재현율 70( 1) 그러나 (0 )이 70 대로 떨어짐
enn = EditedNearestNeighbours()
X_resampled, y_resampled  = enn.fit_resample(X, y)
print(X_resampled.shape)

(1130, 8)


In [41]:
# 재현율 57 
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X, y)
print(X_resampled.shape)

(1138, 7)


In [43]:
from imblearn.under_sampling import NearMiss
nm1 = NearMiss(version=1)
X_resampled, y_resampled = nm1.fit_resample(X, y)
print(X_resampled.shape)

(1138, 7)


In [47]:
renn = RepeatedEditedNearestNeighbours()
X_resampled, y_resampled = renn.fit_resample(X, y)
print(X_resampled.shape)

(1114, 7)


In [45]:
allknn = AllKNN()
X_resampled, y_resampled = allknn.fit_resample(X, y)
print(X_resampled.shape)

(1029, 7)


In [49]:
#언더 샘플링 중에는 제일 좋네 그나마
cnn = CondensedNearestNeighbour(random_state=0)
X_resampled, y_resampled = cnn.fit_resample(X, y)
print(X_resampled.shape)

(960, 7)


In [51]:
# 니가 일등이네
oss = OneSidedSelection(random_state=0)
X_resampled, y_resampled = oss.fit_resample(X, y)
print(X_resampled.shape)

(1544, 7)


In [53]:
ncr = NeighbourhoodCleaningRule()
X_resampled, y_resampled = ncr.fit_resample(X, y)
print(X_resampled.shape)

(1109, 7)


In [ ]:
from sklearn.linear_model import LogisticRegression
iht = InstanceHardnessThreshold(random_state=0,                                 estimator=LogisticRegression(
                                    solver='lbfgs', multi_class='auto'))
X_resampled, y_resampled = iht.fit_resample(X_train, y_train)
print(X_resampled.shape)

#### 오버샘플링 smotenc / smoten

In [73]:
# 재현율 57 프로 나머지, 평균 80
from imblearn.over_sampling import SMOTENC
smote_nc = SMOTENC(categorical_features=[0, 2], random_state=0)
X_resampled, y_resampled = smote_nc.fit_resample(X, y)
print(X_resampled.shape)

(2040, 8)


In [57]:
from imblearn.over_sampling import SMOTEN
sampler = SMOTEN(random_state=0)
X_resampled, y_resampled = sampler.fit_resample(X, y)
print(X_resampled.shape)

(2040, 7)


#### 오버와 언더 조합

In [59]:
from imblearn.combine import *
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
print(X_resampled.shape)

(1078, 7)


In [70]:
smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_resample(X, y)
print(X_resampled.shape)

(1978, 8)


#### 분류 모델

In [71]:
from sklearn import tree
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)

tree_model.fit(X_resampled, y_resampled)
y_hat = tree_model.predict(X_test)
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)


[[249   8]
 [ 59  82]]
              precision    recall  f1-score   support

           0       0.81      0.97      0.88       257
           1       0.91      0.58      0.71       141

    accuracy                           0.83       398
   macro avg       0.86      0.78      0.80       398
weighted avg       0.84      0.83      0.82       398



In [27]:
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='auto',
                                replacement=False,
                                random_state=0)
bbc.fit(X_resampled, y_resampled) 
y_hat = bbc.predict(X_test)
bbc_matrix = metrics.confusion_matrix(y_test, y_hat)
print(bbc_matrix)
bbc_report = metrics.classification_report(y_test, y_hat)
print(bbc_report)

[[225  32]
 [ 55  86]]
              precision    recall  f1-score   support

           0       0.80      0.88      0.84       257
           1       0.73      0.61      0.66       141

    accuracy                           0.78       398
   macro avg       0.77      0.74      0.75       398
weighted avg       0.78      0.78      0.78       398



In [28]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
brf.fit(X_resampled, y_resampled) 
y_hat = brf.predict(X_test)
brf_matrix = metrics.confusion_matrix(y_test, y_hat)
print(brf_matrix)
brf_report = metrics.classification_report(y_test, y_hat)
print(brf_report)

[[227  30]
 [ 54  87]]
              precision    recall  f1-score   support

           0       0.81      0.88      0.84       257
           1       0.74      0.62      0.67       141

    accuracy                           0.79       398
   macro avg       0.78      0.75      0.76       398
weighted avg       0.79      0.79      0.78       398



In [29]:
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(random_state=0)
eec.fit(X_resampled, y_resampled)
y_hat = eec.predict(X_test)
eec_matrix = metrics.confusion_matrix(y_test, y_hat)
print(eec_matrix)
eec_report = metrics.classification_report(y_test, y_hat)
print(eec_report)

[[236  21]
 [ 58  83]]
              precision    recall  f1-score   support

           0       0.80      0.92      0.86       257
           1       0.80      0.59      0.68       141

    accuracy                           0.80       398
   macro avg       0.80      0.75      0.77       398
weighted avg       0.80      0.80      0.79       398



#### 랜덤포레스트

In [72]:
#랜덤 포레스트는 오버샘플링이 좋음
# 오버 언더 조합도 낫베드( 데이터 개수가 늘어날 수록 좋음)
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_resampled , y_resampled)
y_hat = rf_clf.predict(X_test)
rf_clf_matrix = metrics.confusion_matrix(y_test, y_hat)
print(rf_clf_matrix)
rf_clf_report = metrics.classification_report(y_test, y_hat)
print(rf_clf_report)

[[229  28]
 [ 53  88]]
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       257
           1       0.76      0.62      0.68       141

    accuracy                           0.80       398
   macro avg       0.79      0.76      0.77       398
weighted avg       0.79      0.80      0.79       398



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=10)
from sklearn import tree
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)

# train data를 가지고 모형 학습m
tree_model.fit(X_train, y_train)
y_hat = tree_model.predict(X_test)
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

In [ ]:
df_Xtrain = pd.DataFrame(X_train)
df_ytrain = pd.DataFrame(y_train)
df_Xtrain = pd.concat([df_Xtrain, df_ytrain], axis = 1)
is_zero = (df_Xtrain["FrequentFlyer"]==0)&(df_Xtrain["EverTravelledAbroad"]==0)&(df_Xtrain["TravelInsurance"]==1)
df_Xtrain['is_zero'] = is_zero
df_Xtrain["is_zero"] = df_Xtrain["is_zero"].map({False : 0, True : 1})
df_Xtrain = df_Xtrain[df_Xtrain['is_zero'] != 1]
df_Xtrain = df_Xtrain.drop("is_zero" , axis=1)
ndf1 = df_Xtrain[['Age', 'AnnualIncome', 'ChronicDiseases', 
          'FrequentFlyer', 'FamilyMembers', 'Employment Type',
          'EverTravelledAbroad', 'TravelInsurance']]

X_resampled  = ndf1[list(ndf1.columns)[:-1]]
y_resampled  = ndf1[list(ndf1.columns)[-1]]

df_Xtrain.info()